### **1. Authentication**

#### **1.1. Basic Authentication Mechanism**
- Authentication is the process of verifying a user's identity.
- It is critical in client-server interactions.
- Authentication data is sent to the server via HTTP headers.

#### **1.2. HTTP Headers for Authentication**
- HTTP headers play a vital role in data exchange between the client and server.
- The `Authorization` header carries the authentication credentials.
- Example: `Authorization: Basic <credentials>`
  - `<credentials>` is the Base64-encoded string of "username:password".

#### **1.3. Implementing Basic Authentication in FastAPI**

```python
# main_01.py
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import HTTPBasic, HTTPBasicCredentials

app = FastAPI()
security = HTTPBasic()

def get_current_username(credentials: HTTPBasicCredentials = Depends(security)):
    if credentials.username != "william" or credentials.password != "password":
        raise HTTPException(status_code=401, detail="Unauthorized")
    return credentials.username

@app.get("/users/me")
def read_current_user(username: str = Depends(get_current_username)):
    return {"username": username}
# To run: uvicorn main_01:app --reload
```

- **HTTPBasic** implements the basic authentication method.
- The `get_current_username` function validates the username and password.
- If the credentials are incorrect, a 401 Unauthorized error is raised.
- The `/users/me` endpoint is accessible only to authenticated users.

<img src='../source/img/13/01/01.png' width='80%'>

> **Example Command:**  
> `curl -X 'GET' "http://127.0.0.1:8000/users/me" -u william:password`  
>  
> Using this basic authentication setup in FastAPI, the server verifies that the request contains valid authentication data and responds accordingly.

### **2. Session**

#### **2.1. Role of Sessions**
- Sessions are essential for maintaining state in web applications.
- They address the stateless nature of the HTTP protocol.

#### **2.2. How Sessions Work**
- Upon the first request, the server issues a unique session ID to the client.
- The session ID is stored in a client-side cookie.
- On subsequent requests, the cookie containing the session ID is sent to the server.
- The server uses the session ID to identify the client and either store or retrieve session data.

### **3. Using Sessions in FastAPI**

#### **3.1. Utilizing FastAPI with Starlette Sessions**
- FastAPI inherits session management capabilities from Starlette.
- The `SessionMiddleware` provided by Starlette handles session management.

#### **3.2. Configuring Session Middleware**
- Add the `SessionMiddleware` to your FastAPI application to enable session support.
- The `secret_key` is used to encrypt session data and must be kept secure.

```python
# main_02.py
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your_secret_key")
```

#### **3.3. Managing the `secret_key`**
- Use environment variables to securely manage the secret key.
- Ensure that the key is sufficiently long and complex.
- Consider using key management services (e.g., AWS Secrets Manager, HashiCorp Vault) if needed.
- Regularly rotate the key as part of good security practices.

#### **3.4. Setting and Retrieving Session Data**
- Use the `Request` object to store and retrieve data from the session.

```python
# main_02.py
@app.post("/set/")
async def set_session(request: Request):
    request.session["username"] = "william"
    return {"msg": "Session value set"}
```

```python
# main_02.py
@app.get("/get/")
async def get_session(request: Request):
    username = request.session.get("username", "Guest")
    return {"username": username}
```

#### **3.5. Session Retrieval and Testing Example**

```python
# main_02.py
from fastapi import FastAPI, Request
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your-secret-key")

@app.post("/set/")
async def set_session(request: Request):
    request.session["username"] = "william"
    return {"message": "Session value set"}

@app.get("/get/")
async def get_session(request: Request):
    # Retrieve the 'username' from the session. If not found, default to "Guest".
    username = request.session.get("username", "Guest")
    return {"username": username}
# To run: uvicorn main_02:app --reload
```

#### **3.6. Testing Sessions with curl**

1. **Set Session Request**  
   Use the `-c cookies.txt` flag to store cookies in a file.

   ```bash
   curl -X "POST" "http://127.0.0.1:8000/set/" -i -c cookies.txt
   ```

2. **Get Session Request**  
   Use the `-b cookies.txt` flag to send cookies from the file with your request.

   ```bash
   curl -X "GET" "http://127.0.0.1:8000/get/" -i -b cookies.txt
   ```

> **Additional Explanation: Cookie File Structure and Flags**  
> 
> - **Cookie File Structure:**  
>   The cookie file (`cookies.txt`) generated by curl contains details for each cookie, such as:
>   - **Domain:** The domain for which the cookie is valid.
>   - **Flag:** Indicates whether the cookie is valid for subdomains.
>   - **Path:** The URL path for which the cookie is valid.
>   - **Secure:** Indicates if the cookie should only be transmitted over secure connections.
>   - **Expiration:** The expiration time of the cookie in UNIX timestamp format.
>   - **Name:** The name of the cookie.
>   - **Value:** The cookie's value.
>   - Example line:  
>     `example.com    FALSE    /    TRUE    0    SESSIONID    abc123`
> 
> - **Cookie Flags in curl:**  
>   - `-c cookies.txt`: Saves cookies from the server response into `cookies.txt`.
>   - `-b cookies.txt`: Loads cookies from `cookies.txt` and sends them with the request.
>   These flags help maintain session state across multiple requests.

### **4. FastAPI Login Example Using Sessions and Testing**

#### **4.1. Session Middleware Setup**
- Add `SessionMiddleware` to the FastAPI app to enable session management.
- The `secret_key` is used to encrypt session data.

#### **4.2. Login Endpoint (`/login/`)**
- Validates the provided `username` and `password`.
- If the credentials are valid, stores the username in the session.

#### **4.3. Dashboard Endpoint (`/dashboard/`)**
- Retrieves the username from the session.
- Denies access if the user is not logged in.

```python
# main_03.py
from fastapi import FastAPI, Request, HTTPException
from starlette.middleware.sessions import SessionMiddleware

app = FastAPI()

# Add session middleware; replace 'your_secret_key' with a secure key in production.
app.add_middleware(SessionMiddleware, secret_key="your_secret_key")

@app.post("/login/")
async def login(request: Request, username: str, password: str):
    if username == "william" and password == "1234":
        request.session["username"] = username
        return {"message": "Successfully logged in"}
    else:
        raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/dashboard/")
async def dashboard(request: Request):
    username = request.session.get("username")
    if not username:
        raise HTTPException(status_code=401, detail="Not authorized")
    return {"message": f"Welcome to the dashboard, {username}"}
# To run: uvicorn main_03:app --reload
```

#### **4.4. Testing the Login and Dashboard Endpoints**

1. **Login Test**  
   Attempt to log in and store the session cookie in a file using the `-c cookies.txt` flag:

   ```bash
   curl -X "POST" "http://127.0.0.1:8000/login/?username=william&password=1234" -i -c cookies.txt
   ```

2. **Dashboard Access Test**  
   After logging in, access the dashboard using the stored session cookie with the `-b cookies.txt` flag:

   ```bash
   curl -X "GET" "http://127.0.0.1:8000/dashboard/" -i -b cookies.txt
   ```